# Task 0: Introduction

The MNIST dataset is a benchmark dataset in machine learning and computer vision, consisting of 70,000 grayscale images of handwritten digits (0–9), each of size 28x28 pixels. The task is to classify each image into its corresponding digit, making it a multi-class classification problem. This challenge serves as an excellent introduction to deep learning, enabling the application of neural networks to solve real-world problems.

In this notebook, we will build a neural network using PyTorch to classify the MNIST digits. The solution involves loading and preprocessing the data, designing and training a deep learning model, evaluating its performance, and generating predictions for submission to Kaggle. This structured approach ensures reproducible results and facilitates understanding of fundamental deep learning concepts.

# Task 1: Importing Libraries

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Importing Necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim

# Task 2: Loading the Dataset

In [ ]:
# Load the MNIST Dataset
train_data_path = "/kaggle/input/digit-recognizer/train.csv"
test_data_path = "/kaggle/input/digit-recognizer/test.csv"

# Load Training Data
train_df = pd.read_csv(train_data_path)

# Task 3: Exploratory Data Analysis (EDA)

In [ ]:
# Exploratory Data Analysis
# Display some basic statistics
print(train_df.describe())

In [ ]:
# Visualize a few samples from the dataset
def visualize_samples(dataframe, num_samples=6):
    """Visualizes random samples from the dataset."""
    samples = dataframe.sample(num_samples)
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 4))
    for i, (idx, row) in enumerate(samples.iterrows()):
        label = row['label']
        image = row.drop('label').values.reshape(28, 28)
        axes[i].imshow(image, cmap='gray')
        axes[i].axis('off')
        axes[i].set_title(f"Label: {label}")
    plt.show()

visualize_samples(train_df)

# Task 4: Data Preprocessing

In [ ]:
# Data Preprocessing with a Custom Dataset Class
class MNISTDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.labels = dataframe['label'].values
        self.images = dataframe.drop(columns=['label']).values.astype(np.float32) / 255.0

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx].reshape(28, 28)
        label = self.labels[idx]
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Task 5: Splitting Data into Training and Validation Sets

In [ ]:
# Train-Validation Split
train_set, val_set = train_test_split(train_df, test_size=0.2, random_state=42)
train_dataset = MNISTDataset(train_set)
val_dataset = MNISTDataset(val_set)

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Task 6: Building the Neural Network Model

In [ ]:
# Define the Neural Network Model
input_layer = 784
hidden_layer1 = 128
hidden_layer2 = 64
output_layer = 10

model = nn.Sequential(
    nn.Linear(input_layer, hidden_layer1),
    nn.ReLU(),
    nn.Linear(hidden_layer1, hidden_layer2),
    nn.ReLU(),
    nn.Linear(hidden_layer2, output_layer)
)

# Loss Function and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Task 7: Training the Model

In [ ]:
# Training the Model
def train_model(model, train_loader, val_loader, epochs=10):
    """Trains the neural network and evaluates it on validation data."""
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images = images.view(images.size(0), -1)
            optimizer.zero_grad()
            predictions = model(images)
            loss = loss_function(predictions, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Validation Step
        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(images.size(0), -1)
                predictions = model(images)
                val_loss += loss_function(predictions, labels).item()
                correct += (predictions.argmax(1) == labels).sum().item()
        
        # Print Epoch Metrics
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {total_loss/len(train_loader):.4f}")
        print(f"Validation Loss: {val_loss/len(val_loader):.4f}")
        print(f"Validation Accuracy: {100 * correct / len(val_dataset):.2f}%")
        print("-" * 30)

In [ ]:
train_model(model, train_loader, val_loader)

# Task 8: Preprocessing Test Data

In [ ]:
# Evaluate on Test Data
test_df = pd.read_csv(test_data_path)

In [ ]:
# Prepare Test Dataset
class MNISTTestDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.images = dataframe.values.astype(np.float32) / 255.0

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].reshape(28, 28)
        return torch.tensor(image, dtype=torch.float32)

In [ ]:
test_dataset = MNISTTestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Task 9: Making Predictions

In [ ]:
# Make Predictions
def make_predictions(model, test_loader):
    """Predicts labels for the test dataset."""
    model.eval()
    predictions = []
    with torch.no_grad():
        for images in test_loader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            predicted_labels = outputs.argmax(1).tolist()
            predictions.extend(predicted_labels)
    return predictions

In [ ]:
test_predictions = make_predictions(model, test_loader)

# Task 10: Creating the Submission File

In [ ]:
# Save Results to Submission File
submission = pd.DataFrame({
    'ImageId': range(1, len(test_predictions) + 1),
    'Label': test_predictions
})
submission.to_csv("submission.csv", index=False)
print("Submission file created: submission.csv")